## List comprehensions: Database integrations
### Extract data from database tables

In [1]:
import e3_queries

print(dir(e3_queries))

['SQL_CHART_DATA_BY_SWIMMER_EVENT_SESSION', 'SQL_SESSIONS', 'SQL_SWIMMERS_BY_SESSION', 'SQL_SWIMMERS_EVENTS_BY_SESSION', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__']


In [2]:
# test
statements = []

for item in dir(e3_queries):
    if not item.startswith("__"):
        statements.append(item)

statements

['SQL_CHART_DATA_BY_SWIMMER_EVENT_SESSION',
 'SQL_SESSIONS',
 'SQL_SWIMMERS_BY_SESSION',
 'SQL_SWIMMERS_EVENTS_BY_SESSION']

In [3]:
# list comprehension
[item for item in dir(e3_queries) if not item.startswith("__")]

['SQL_CHART_DATA_BY_SWIMMER_EVENT_SESSION',
 'SQL_SESSIONS',
 'SQL_SWIMMERS_BY_SESSION',
 'SQL_SWIMMERS_EVENTS_BY_SESSION']

In [4]:
import DBcm

db_details = "CoachDB.sqlite3"

from e3_queries import SQL_SESSIONS
print(SQL_SESSIONS)


    select distinct ts from times



In [5]:
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_SESSIONS)
    results = db.fetchall()

results

[('2023-09-14 21:41:01',), ('2023-09-14 21:41:02',)]

In [6]:
from e3_queries import SQL_SWIMMERS_BY_SESSION

print(SQL_SWIMMERS_BY_SESSION)


    select distinct swimmers.name, swimmers.age
    from times, swimmers
    where date(times.ts) = ? and
    times.swimmer_id = swimmers.id
    order by name



In [7]:
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_SWIMMERS_BY_SESSION, ("2023-09-14",))
    results = db.fetchall()

results

[('Abi', 10),
 ('Ali', 12),
 ('Alison', 14),
 ('Aurora', 13),
 ('Bill', 18),
 ('Blake', 15),
 ('Calvin', 9),
 ('Carl', 15),
 ('Chris', 17),
 ('Darius', 13),
 ('Dave', 17),
 ('Elba', 14),
 ('Emma', 13),
 ('Erika', 15),
 ('Hannah', 13),
 ('Katie', 9),
 ('Lizzie', 14),
 ('Maria', 9),
 ('Mike', 15),
 ('Owen', 15),
 ('Ruth', 13),
 ('Tasmin', 15)]

In [8]:
from e3_queries import SQL_SWIMMERS_EVENTS_BY_SESSION

print(SQL_SWIMMERS_EVENTS_BY_SESSION)


    select distinct events.distance, events.stroke
    from swimmers, events, times
    where times.swimmer_id = swimmers.id and
    times.event_id = events.id and
    (swimmers.name = ? and swimmers.age = ?) and
    date(times.ts) = ?



In [9]:
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_SWIMMERS_EVENTS_BY_SESSION, ("Darius", 13, "2023-09-14",))
    results = db.fetchall()

results

[('100m', 'Back'), ('100m', 'Breast'), ('100m', 'Fly'), ('200m', 'IM')]

In [10]:
from e3_queries import SQL_CHART_DATA_BY_SWIMMER_EVENT_SESSION

print(SQL_CHART_DATA_BY_SWIMMER_EVENT_SESSION)


    select times.time
    from swimmers, events, times
    where (swimmers.name = ? and swimmers.age = ?) and
    (events.distance = ? and events.stroke = ?) and
    swimmers.id = times.swimmer_id and
    events.id = times.event_id and
    date(times.ts) = ?



In [11]:
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_CHART_DATA_BY_SWIMMER_EVENT_SESSION, 
               ("Darius", 13, "100m", "Fly", "2023-09-14",))
    results = db.fetchall()

results

[('1:27.95',),
 ('1:21.07',),
 ('1:30.96',),
 ('1:23.22',),
 ('1:27.95',),
 ('1:28.30',)]

#### Test data_utils.py

In [12]:
import e3_data_utils

print([x for x in dir(e3_data_utils) if not x.startswith("__")])

['DBcm', 'SQL_CHART_DATA_BY_SWIMMER_EVENT_SESSION', 'SQL_SESSIONS', 'SQL_SWIMMERS_BY_SESSION', 'SQL_SWIMMERS_EVENTS_BY_SESSION', 'db_details', 'get_session_swimmers', 'get_swim_session', 'get_swimmers_events', 'get_swimmers_times']


In [13]:
help(e3_data_utils.get_swim_session)

Help on function get_swim_session in module e3_data_utils:

get_swim_session()
    Return a tuple-list of unique session timestamps.



In [14]:
data = e3_data_utils.get_swim_session()
data

[('2023-09-14 21:41:01',), ('2023-09-14 21:41:02',)]

In [15]:
[session[0].split(" ")[0] for session in data]

['2023-09-14', '2023-09-14']

In [16]:
data = e3_data_utils.get_session_swimmers("2023-09-14")
data

[('Abi', 10),
 ('Ali', 12),
 ('Alison', 14),
 ('Aurora', 13),
 ('Bill', 18),
 ('Blake', 15),
 ('Calvin', 9),
 ('Carl', 15),
 ('Chris', 17),
 ('Darius', 13),
 ('Dave', 17),
 ('Elba', 14),
 ('Emma', 13),
 ('Erika', 15),
 ('Hannah', 13),
 ('Katie', 9),
 ('Lizzie', 14),
 ('Maria', 9),
 ('Mike', 15),
 ('Owen', 15),
 ('Ruth', 13),
 ('Tasmin', 15)]

In [17]:
print([f"{swimmer[0]} - {swimmer[1]}" for swimmer in data])

['Abi - 10', 'Ali - 12', 'Alison - 14', 'Aurora - 13', 'Bill - 18', 'Blake - 15', 'Calvin - 9', 'Carl - 15', 'Chris - 17', 'Darius - 13', 'Dave - 17', 'Elba - 14', 'Emma - 13', 'Erika - 15', 'Hannah - 13', 'Katie - 9', 'Lizzie - 14', 'Maria - 9', 'Mike - 15', 'Owen - 15', 'Ruth - 13', 'Tasmin - 15']


In [18]:
e3_data_utils.get_swimmers_events("Owen", 15, "2023-09-14")

[('100m', 'Free')]

In [19]:
data = e3_data_utils.get_swimmers_events("Darius", 13, "2023-09-14")
data

[('100m', 'Back'), ('100m', 'Breast'), ('100m', 'Fly'), ('200m', 'IM')]

In [20]:
[f"{event[0]} {event[1]}" for event in data]

['100m Back', '100m Breast', '100m Fly', '200m IM']

In [22]:
data = e3_data_utils.get_swimmers_times("Owen", 15, "100m", "Free", "2023-09-14")
data

[('1:15.57',), ('1:14.40',), ('1:19.82',), ('1:12.90',)]

In [25]:
[time[0] for time in data]

['1:15.57', '1:14.40', '1:19.82', '1:12.90']